# MLflow Tracking

The MLflow Tracking is an API and UI for logging parameters, code versions, metrics, and output files when running your machine learning code and for later visualizing the results.

## Concepts

![Taken from MLflow Docs](https://mlflow.org/docs/latest/_images/tracking-basics.png)

**Runs**

MLflow Tracking is organized around the concept of runs, which are executions of some piece of data science code, for example, a single python train.py execution.


**Experiments** 

An experiment groups together runs for a specific task. 




In [1]:
import mlflow 
from mlflow_for_ml_dev.utils.utils import get_root_project

In [2]:
print(get_root_project())

C:\Users\manue\projects\mlflow_for_ml_dev


# Create an experiment

In [3]:
artifact_location = get_root_project() / "experiments" / "mlruns"

mlflow.set_tracking_uri(artifact_location.as_uri())
print(artifact_location)

C:\Users\manue\projects\mlflow_for_ml_dev\experiments\mlruns


In [4]:
# Create experiment
experiment_name = "main-concepts-01"
experiment_id = mlflow.create_experiment(name=experiment_name)
print(experiment_id)

266958456446287739


After creating an experiment is necessary to set it as active experiment, otherwise mlflow would use the Default experiment.

In [5]:
with mlflow.start_run() as run:
    # Log some parameters and metrics here
    print(run.info.run_id)

2f5384c7e4ea473093e3f1e6984475ad


In [6]:
# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [7]:
with mlflow.start_run() as run:
    # Log some parameters and metrics here
    print(run.info.run_id)

fd3cff2d692b4b0bb3715118fa38c9dc


An experiment can also be created by using `mlflow.set_experiment(experiment_name)`. If the experiment does not exist mlflow creates an experiment using the provided name. Since the `experiment_name`

In [8]:
# creating an experiment using the same name will raise an exception
experiment_id = mlflow.create_experiment(name=experiment_name)

MlflowException: Experiment 'main-concepts-01' already exists.

# Two Options

## Capture the exception

In [9]:
try: 
    experiment_id = mlflow.create_experiment(name=experiment_name)
except mlflow.exceptions.MlflowException as e:
    print(e)

Experiment 'main-concepts-01' already exists.


## Use `mlflow.set_experiment`

In [10]:
# Since the experiment already exists, we can set it as the active experiment
experiment = mlflow.set_experiment(experiment_name = experiment_name)

# we can also create the experiment if it does not exist
new_experiment_name = "main-concepts-02"
experiment = mlflow.set_experiment(experiment_name = new_experiment_name)

2024/07/11 21:18:30 INFO mlflow.tracking.fluent: Experiment with name 'main-concepts-02' does not exist. Creating a new experiment.


## Adding tags

In [11]:
experiment_name = "main-concepts-03"
experiment_id = mlflow.create_experiment(
    name=experiment_name,
    tags={"topic":"experiment_management", "project_name":"UNKNOWN"}
)

# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [12]:
# get the experiment tags
experiment.tags

{'project_name': 'UNKNOWN', 'topic': 'experiment_management'}

## Adding a description

In [13]:
experiment_name = "main-concepts-04"
experiment_id = mlflow.create_experiment(
    name=experiment_name,
    tags={
        "topic":"experiment_management",
        "project_name":"UNKNOWN",
        "mlflow.note.content":"This is a test experiment"})

In [16]:
# Set the experiment as active experiment
experiment = mlflow.set_experiment(experiment_name)

In [17]:
# get the experiment tags
experiment.tags

{'mlflow.note.content': 'This is a test experiment.  Adding more information',
 'project_name': 'UNKNOWN',
 'topic': 'experiment_management'}

## Update Tags

In [18]:
experiment_name = "main-concepts-04"
experiment = mlflow.set_experiment(experiment_name)
# experiment tags
experiment.tags

{'mlflow.note.content': 'This is a test experiment.  Adding more information',
 'project_name': 'UNKNOWN',
 'topic': 'experiment_management'}

In [19]:
tags = {
    "tag1": "value1",
    "tag2": "value2"
}
mlflow.set_experiment_tags(tags=tags)


In [20]:
# get the updated experiment object
experiment = mlflow.set_experiment(experiment_name)

# get the experiment tags
experiment.tags

{'mlflow.note.content': 'This is a test experiment.  Adding more information',
 'project_name': 'UNKNOWN',
 'tag1': 'value1',
 'tag2': 'value2',
 'topic': 'experiment_management'}

In [21]:
# Update Value of tag1
mlflow.set_experiment_tag(key="tag1", value="new_value1")

# get the updated experiment object
experiment = mlflow.set_experiment(experiment_name)

In [22]:
# get the experiment tags
experiment.tags

{'mlflow.note.content': 'This is a test experiment.  Adding more information',
 'project_name': 'UNKNOWN',
 'tag1': 'new_value1',
 'tag2': 'value2',
 'topic': 'experiment_management'}

## Using the client to set a tag

In [23]:
client = mlflow.MlflowClient()

In [24]:
experiment.experiment_id

'528269083760830486'

In [25]:
client.set_experiment_tag(experiment_id = experiment.experiment_id, key="tag3", value="value3")

In [26]:
experiment = mlflow.set_experiment(experiment_name)

# get the experiment tags
experiment.tags

{'mlflow.note.content': 'This is a test experiment.  Adding more information',
 'project_name': 'UNKNOWN',
 'tag1': 'new_value1',
 'tag2': 'value2',
 'tag3': 'value3',
 'topic': 'experiment_management'}

## Rename Experiment

In [27]:
new_name = "main-concepts-04-renamed"
client.rename_experiment(experiment_id = experiment.experiment_id, new_name=new_name)

In [28]:
experiment = mlflow.set_experiment(new_name)

experiment.name

'main-concepts-04-renamed'

## Clean Up

In [29]:
experiments = mlflow.search_experiments(filter_string="name LIKE 'main-concepts%'")

In [31]:
for exp in experiments:
    print(exp.experiment_id, exp.name, exp.tags)

528269083760830486 main-concepts-04-renamed {'mlflow.note.content': 'This is a test experiment.  Adding more information', 'project_name': 'UNKNOWN', 'tag1': 'new_value1', 'tag2': 'value2', 'tag3': 'value3', 'topic': 'experiment_management'}
827829603635428632 main-concepts-03 {'project_name': 'UNKNOWN', 'topic': 'experiment_management'}
682414674497756855 main-concepts-02 {}
266958456446287739 main-concepts-01 {}


In [32]:
for experiment in experiments:
    print(f"Deleting: {experiment.name}")
    mlflow.delete_experiment(experiment.experiment_id)

Deleting: main-concepts-04-renamed
Deleting: main-concepts-03
Deleting: main-concepts-02
Deleting: main-concepts-01
